# snowmobile

```{include} /description.md
```

```{eval-rst}

.. toctree::
    :maxdepth: 1
    :hidden:

    ./setup.md

.. toctree::
    :caption: Core
    :maxdepth: 1
    :hidden:

    ./usage/snowmobile.md
    ./usage/script.md
    ./usage/table.md
    ./usage/sql.ipynb
    ./usage/snowmobile_toml.md

.. toctree::
    :caption: Technical Resources
    :maxdepth: 1
    :hidden:

    ./autoapi/snowmobile/core/index
    ./snippets.md

.. toctree::
    :caption: Other
    :maxdepth: 1
    :hidden:

    ./acknowledgements.md
    ./changelog.md
    ./authors.md
    ./license.md
```

+++

## Overview

```{div} sn-dedent-list, sn-depth1
- *[Connecting](#connecting)*
- *[Query Execution](#query-execution)*
- *[Information API](#information-api)*
- *[Loading Data](#loading-data)*
- *[Working with SQL Scripts](#working-with-sql-scripts)*
```

<br>
<br>

```{div} sn-phantom
&nbsp;
```
### *Connecting*

````{admonition} *Connecting*
:class: toggle, sn-gradient-header, sn-indent-h-cell-right-m, sn-toggle-expand

```{div} sn-dedent-v-b-h, sn-dedent-v-t-container-neg
{func}`snowmobile.connect()` returns a [Snowmobile](./usage/snowmobile.md#snowmobile) whose purpose is to:
```
1.  Locate, instantiate, and store [snowmobile.toml](./usage/snowmobile_toml.md#snowmobiletoml)
    as a {class}`~snowmobile.Configuration` object ({class}`sn.cfg`)
1.  Establish a connection to {xref}`snowflake` and store the {xref}`SnowflakeConnection` ({class}`sn.con`)
1.  Serve as the primary entry point to the {xref}`SnowflakeConnection` and {xref}`snowmobile` APIs
+++
```{div} sn-dedent-v-t-h
  The first time it's invoked, [Snowmobile](./usage/snowmobile.md) will find [snowmobile.toml](./usage/snowmobile_toml) and cache its location;
  this step isn't repeated unless the file is moved, the cache is manually cleared, or a new version of {xref}`snowmobile` is installed.
```
+++
**With all arguments omitted, it will authenticate with the default credentials and connection arguments specified in** [**snowmobile.toml**](./usage/snowmobile_toml).

<hr class="sn-blue">

```{div} sn-link-container
{link-badge}`./usage/snowmobile.html,cls=badge-primary text-white sn-usage,Usage: Snowmobile,tooltip=Intro & usage documentation for snowmobile.Snowmobile`
{link-badge}`./autoapi/snowmobile/core/connection/index.html,cls=badge-secondary text-white sn-api,snowmobile.core.connection,tooltip=API Documentation`
```

````

```{div} sn-pre-code
Establishing a connection from configured defaults is done with:
```

# -- SETUP --

In [3]:
# -- setup --

import sys

from pathlib import Path

sys.path.insert(0, str(Path.cwd().parent))

In [4]:
import snowmobile

sn = snowmobile.connect()

Locating credentials..
(1 of 2) Finding snowmobile.toml..
(2 of 2) Cached path found at ../Snowmobile/snowmobile.toml
..connected: snowmobile.Snowmobile(creds='creds1')


```{div} sn-pre-code, sn-post-code
`sn` is a [](./usage/snowmobile) with the following attributes:
```

In [6]:
print(sn)            #> snowmobile.Snowmobile(creds='creds1')
print(sn.cfg)        #> snowmobile.Configuration('snowmobile.toml')
print(type(sn.con))  #> <class 'snowflake.connector.connection.SnowflakeConnection'>

snowmobile.Snowmobile(creds='creds1')
snowmobile.Configuration('snowmobile.toml')
<class 'snowflake.connector.connection.SnowflakeConnection'>


```{div} sn-pre-code, sn-post-code
(fixture-sn)=
Specific connection arguments are accessed by pre-configured alias:
```

In [36]:
sn2 = snowmobile.connect(creds='sandbox')

print(sn.cfg.connection.current != sn2.cfg.connection.current) #> True

In [3]:
# -- setup --

# Verify table doesn't exist
if sn.exists('sample_table'):
    sn.drop('sample_table')

# Truncate 'any_other_table'
sn.truncate('any_other_table')

# Path to sample .sql file
from pathlib import Path

path = Path.cwd() / 'snippets' / 'overview' / 'sample_table.sql'

# Execute setup statements from script
script = snowmobile.Script(sn=sn, path=path)
script.run((1,2))

sample_table.sql
<1 of 5> create table~s1 (0s)......................................... <completed>
<2 of 5> insert into~s2 (0s).......................................... <completed>


<br>

`````{admonition} **sn**
:class: sn-fixture, sn-fixture-global

````{div} sn-dedent-v-b-h, sn-dedent-v-t-container-neg
The variable `sn` represents a generic instance of [](./usage/snowmobile) roughly
equivalent to that created with the snippet below; it's referred to as {fa}`fixture sn` 
throughout the documentation, and applicable examples make use of it as a fixture 
without explicitly re-instantiating.
````

```{code-block} python
:emphasize-lines: 3, 3

import snowmobile

sn = snowmobile.connect()
```

<hr class="sn-spacer">

`````

<hr class="sn-spacer">

```{div} sn-phantom
<br>
```
### *Query Execution*

`````{admonition} *Query Execution*
:class: toggle, sn-gradient-header, sn-indent-h-cell-right-m, sn-toggle-expand

```{div} sn-dedent-v-b-h, sn-dedent-v-container-neg
  [{fa}`fixture sn`](fixture-sn) provides three convenience methods for executing raw SQL:
```

```{div} sn-left-pad
{meth}`~snowmobile.Snowmobile.query()` implements {meth}`pandas.read_sql()` for querying results into a {class}`~pandas.DataFrame`

{meth}`~snowmobile.Snowmobile.ex()` implements {meth}`SnowflakeConnection.cursor().execute()` for executing commands within a {xref}`SnowflakeCursor`

{meth}`~snowmobile.Snowmobile.exd()` implements {meth}`SnowflakeConnection.cursor(DictCursor).execute()` for executing commands within a {xref}`DictCursor`
```

<hr class="sn-blue">

```{div} sn-link-container
{link-badge}`./usage/snowmobile.html#executing-raw-sql,cls=badge-primary text-white sn-usage,Usage: Executing Raw SQL,tooltip=Usage documentation for Executing Raw SQL`
{link-badge}`./autoapi/snowmobile/core/connection/index.html,cls=badge-secondary text-white sn-api,snowmobile.core.connection,tooltip=API Documentation`
```

`````

````{admonition} Setup
:class: is-setup, sn-indent-h-cell-m, sn-indent-h-sub-cell-right, sn-inline-block-container, sn-dedent-v-b-container

(sn)=
Assume a pre-existing \
**sample_table**:
```{div} sn-dedent-v-b-container
|   COL1 |   COL2 |
|-------:|-------:|
|      1 |      1 |
|      2 |      4 |
|      4 |      9 |
```
````

<hr class="sn-spacer">

```{div} sn-pre-code
Into a {class}`~pandas.DataFrame`:
```

In [7]:
sn.query('select * from sample_table')

,col1,col2
0,1,1
1,2,4
2,3,9


```{div} sn-pre-code
Into a {xref}`SnowflakeCursor`:
```

In [91]:
sn.ex('select * from sample_table').fetchall()

[(1, 1), (2, 4), (3, 9)]

```{div} sn-pre-code
Into a {xref}`DictCursor`:
```

In [92]:
sn.exd('select * from sample_table').fetchall()

[{'COL1': 1, 'COL2': 1}, {'COL1': 2, 'COL2': 4}, {'COL1': 3, 'COL2': 9}]

```{div} sn-pre-code
Or to get a single value:
```

In [14]:
sn.query('select count(*) from sample_table', as_scalar=True)

3

```{div} sn-pre-code
{xref}`SnowflakeCursor` and {xref}`DictCursor` are also accessible by passing argumentsto 
{meth}`sn.query()<snowmobile.core.connection.Snowmobile.query()>`:
```

In [3]:
sql = 'select * from sample_table'

cur1, cur2 = (
    sn.ex(sql).fetchall(),
    sn.query(sql, as_cur=True).fetchall()
)
print(cur1 == cur2)     #> True

dcur1, dcur2 = (
    sn.exd(sql).fetchall(),
    sn.query(sql, as_dcur=True).fetchall()
)
print(dcur1 == dcur2)   #> True

```{div} sn-pre-code
Conflicting arguments will results in following error:
```

In [4]:
try:
    sn.query(sql, as_df=True, as_dcur=True)
except ValueError as e:
    print(e)

Only one of ('as_df', 'as_cur', 'as_dcur', 'as_scalar') can evaluate to `True`


```{div} sn-phantom
<br>
```
### *Information API*

`````{admonition} *Information API*
:class: toggle, sn-gradient-header, sn-indent-h-cell-right-m, sn-toggle-expand

```{div} sn-dedent-v-container-neg
  [{fa}`fixture sn`](fixture-sn) inherits everything from a [**SQL**](./usage/sql.ipynb) class that generates and executes raw SQL from inputs; its purpose 
  is to provide a bare bones Python API to query metadata and execute administrative commands against {xref}`snowflake`.
```

<hr class="sn-blue">

```{div} sn-link-container
{link-badge}`./usage/sql.html,cls=badge-primary text-white sn-usage,Usage: SQL,tooltip=Usage documentation for snowmobile.SQL`
{link-badge}`./autoapi/snowmobile/core/sql/index.html,cls=badge-secondary text-white sn-api,snowmobile.core.sql,tooltip=API Documentation`
```

`````

<p class="sn-indent-cell"></p>

```{div} sn-pre-code
Check existence:
```

In [5]:
sn.exists('sample_table')  #> True

True

```{div} sn-pre-code, sn-post-code
Select records:
```

In [93]:
sn.select('sample_table', n=1)

,col1,col2
0,1,1


```{div} sn-pre-code
Query metadata:
```

In [5]:
sn.count('sample_table')                 #> 3
sn.count('sample_table', dst_of='col1')  #> 3
sn.columns('sample_table')               #> ['COL1', 'COL2']

```{div} sn-pre-code
Verify dimensionality:
```

In [5]:
sn.is_distinct('sample_table', field='col1')  #> True

```{div} sn-pre-code, sn-post-code
Submit basic administrative commands:
```

In [3]:
sn.clone(nm='sample_table', to='sample_table2')

,status
0,Table SAMPLE_TABLE2 successfully created.


```{div} sn-pre-code, sn-post-code
Fetch DDL:
```

In [53]:
print(sn.ddl('sample_table'))

create or replace TABLE SAMPLE_TABLE (
	COL1 FLOAT,
	COL2 FLOAT
);


```{div} sn-pre-code
Provide `run=False` to get the raw sql
```

In [7]:
drop_sql = sn.drop('sample_table', run=False)

print(drop_sql)        #> drop table if exists GEM7318.SAMPLE_TABLE

<class 'str'>
drop table if exists GEM7318.SAMPLE_TABLE


```{div} sn-pre-code
Drop objects:
```

In [8]:
for t in ['sample_table', 'sample_table2']:
    sn.drop(t, obj='table')

sn.exists('sample_table')   #> False
sn.exists('sample_table2')  #> False

```{div} sn-phantom
<br>
```
### *Loading Data*

(intro/loading-data)=
`````{admonition} *Loading Data*
:class: toggle, sn-gradient-header, sn-indent-h-cell-right-m, sn-toggle-expand, sn-incl-tabbed-shadow-b-blue

<hr class="sn-spacer">

````{tabbed} -
  {class}`~snowmobile.Table` is a loading solution that at minimum accepts a `df` ({class}`~pandas.DataFrame`)
  and a `table` name ({class}`str`).
+++
  In the same way that [**Snowmobile**](./usage/snowmobile) handles its keyword arguments,
  {class}`~snowmobile.Table` will adhere to any arguments explicitly provided and defer
  to the values configured in [snowmobile.toml](./usage/snowmobile_toml) otherwise.
````

````{tabbed} More Info
  ```{div} sn-dedent-v-t
  *The behavior outlined below reflects those within the
  [default snowmobile.toml file](./usage/snowmobile_toml.md#file-contents)*, meaning that `t1` will:
  ```
  1. Check if *sample_table* exists in the schema associated with {attr}`sn.con`
  2. If *sample_table* **does** exist, it will validate `df` against *sample_table* and throw an error
     if their dimensions are not identical
  3. If *sample_table* does **not** exist (as is the case here), it will generate DDL from `df` and execute it as part of the loading process
````

```{div} sn-link-container
{link-badge}`./usage/table.html,cls=badge-primary text-white sn-usage,Usage: snowmobile.Table,tooltip=Intro & usage documentation for snowmobile.Table`
{link-badge}`./autoapi/snowmobile/core/table/index.html,cls=badge-secondary text-white sn-api,snowmobile.core.table,tooltip=API Documentation`
```

``````

`````{admonition} Setup
:class: is-setup, sn-block-container, sn-indent-h-cell-m, sn-indent-h-sub-cell-right

````{panels}

Assume *sample_table* does not \
yet exist and `df` was created with:

```python
import pandas as pd
import numpy as np

df = pd.DataFrame(
    data = {'COL1': [1, 2, 3], 'COL2': [1, 4, 9]}
)
print(df.shape)  #> (3, 2)
```

---

|   COL1 |   COL2 |
|-------:|-------:|
|      1 |      1 |
|      2 |      4 |
|      3 |      9 |

````

`````

In [34]:
# Verify table does not exist before moving forward with example
if sn.exists('sample_table'):
    sn.drop('sample_table')

In [35]:
"""Generate dummy DataFrame for snowmobile.Table example."""
import pandas as pd
import numpy as np

df = pd.DataFrame(
    data = {'COL1': [1, 2, 3], 'COL2': [1, 4, 9]}
)
df.head()

,COL1,COL2
0,1,1
1,2,4
2,3,9


```{div} sn-pre-code, sn-indent-v-t-container, sn-ov-indent
Given [{fa}`cog`](#loading-data), instantiating the following [**Table**](./usage/table), `t1`,
with *as_is=True* will:
1. Generate and execute DDL to create *sample_table*
1. Load `df` into *sample_table* via the {xref}`bulk loading from a local file system` standard
```

In [36]:
import snowmobile

t1 = snowmobile.Table(
    df=df,
    table='sample_table',
    as_is=True,
)

Locating credentials..
(1 of 2) Finding snowmobile.toml..
(2 of 2) Cached path found at ../Snowmobile/snowmobile.toml
..connected: snowmobile.Snowmobile(creds='creds1')
Loading into 'gem7318.SAMPLE_TABLE`..
(1 of 4)
	CREATE OR REPLACE TABLE SAMPLE_TABLE ( ..
(2 of 4)
	create stage SAMPLE_TABLE_stage file_format = snowmobile_default_psv;
(3 of 4)
	put file://C:/Users/gem7318/Documents/Github/Snowmobile/docs/sample_table.csv @SAMPLE_TABLE_stage
	auto_compress = true
(4 of 4)
	copy into SAMPLE_TABLE from @SAMPLE_TABLE_stage
	on_error = continue
..completed: 3 rows in 2 seconds


```{div} sn-pre-code, sn-post-code
In absence of providing an existing [{fa}`fixture sn`](fixture-sn) to {class}`snowmobile.Table()<snowmobile.core.table.Table>`,
an instance was created and stored as a public attribute; the create and load for *sample_table* can be verified with either of:
```

In [8]:
print(t1.sn.exists('sample_table'))  #> True
print(t1.loaded)                     #> True

True
True


```{div} sn-indent-h-cell-m
<hr class="sn-green-medium" style="margin-top: 1.4rem; margin-bottom: 0.5rem;">
```

```{div} sn-pre-code, sn-post-code
When compatability between the *df* and the *table* is unknown, *as_is=True* can be omitted and 
the [**Table**](./usage/table) that's returned inspected further prior to continuing with the loading process:
```

In [37]:
df2 = pd.concat([df, df], axis=1)
print(list(df2.columns))              #> ['COL1', 'COL2', 'COL1', 'COL2']
print(t1.sn.columns('sample_table'))  #> ['COL1', 'COL2']

t2 = snowmobile.Table(
    df=df2,
    table='sample_table',
)

['COL1', 'COL2', 'COL1', 'COL2']
['COL1', 'COL2']
Locating credentials..
(1 of 2) Finding snowmobile.toml..
(2 of 2) Cached path found at ../Snowmobile/snowmobile.toml
..connected: snowmobile.Snowmobile(creds='creds1')


```{div} sn-pre-code, sn-post-code
Primary compatability checks are centered around things like:
```

In [24]:
print(t2.exists)      #> True
print(t2.cols_match)  #> False

True
False
{0: ('col1', 'col1'), 1: ('col2', 'col2'), 2: (None, 'col1_1'), 3: (None, 'col2_1')}


```{div} sn-pre-code
With [snowmobile.toml](./usage/snowmobile_toml) defaults, calling {meth}`Table.load()<snowmobile.core.table.Table.load()>` on `t2` will throw an error:
```

In [33]:
from snowmobile.core.errors import ColumnMismatchError

try:
    t2.load()
except ColumnMismatchError as e:
    print(e)
"""
>>>
ColumnMismatchError: `SAMPLE_TABLE` columns do not equal those in the local DataFrame and 
if_exists='append' was specified.

Either provide if_exists='replace' to overwrite the existing table or see `table.col_diff()`
to inspect the mismatched columns.
"""

ColumnMismatchError: `SAMPLE_TABLE` columns do not equal those in the local DataFrame and if_exists='append' was specified.
Either provide if_exists='replace' to overwrite the existing table or see `table.col_diff` to inspect the mismatched columns.

```{div} sn-pre-code, sn-post-code
{meth}`Table.col_diff()<snowmobile.core.table.Table.col_diff()>` returns a dictionary of tuples containing the table 
and DataFrame columns by index position; providing *mismatch=True* limits the results to only those responsible 
for the {class}`~snowmobile.core.errors.ColumnMismatchError`:
```

In [48]:
import json

print(json.dumps(t2.col_diff(mismatched=True), indent=4))
"""
>>>
{
  "3": [
    null,
    "col1_1"
  ],
  "4": [
    null,
    "col2_1"
  ]
}
"""

{
    "1": [
        "col1",
        "col1"
    ],
    "2": [
        "col2",
        "col2"
    ],
    "3": [
        null,
        "col1_1"
    ],
    "4": [
        null,
        "col2_1"
    ]
}


```{div} sn-pre-code, sn-post-code
Keyword arguments take precedent over configurations in [snowmobile.toml](./usage/snowmobile_toml), so `df2` can still be loaded with:
```

In [79]:
t2.load(if_exists='replace')

print(t2.loaded)                      #> True
print(t2.sn.columns('sample_table'))  #> ['COL1', 'COL2', 'COL1_1', 'COL2_1']

Loading into 'gem7318.SAMPLE_TABLE`..
(1 of 4)
	CREATE OR REPLACE TABLE SAMPLE_TABLE ( ..
(2 of 4)
	create stage SAMPLE_TABLE_stage file_format = snowmobile_default_psv;
(3 of 4)
	put file://C:/Users/GEM7318/Documents/Github/Snowmobile/docs/sample_table.csv @SAMPLE_TABLE_stage
	auto_compress = true
(4 of 4)
	copy into SAMPLE_TABLE from @SAMPLE_TABLE_stage
	on_error = continue
..completed: 3 rows in 2 seconds
True
3
['COL1', 'COL2', 'COL1_1', 'COL2_1']


<hr class="sn-spacer">

`````{div} sn-indent-h-cell, sn-dedent-v-t-container
 ````{note}
 With default behavior, the duplicate column names in `df2` were automatically renamed
 by `t2` before loading into *sample_table*:
 
  ```python
  print(list(df2.columns))    #> ['COL1', 'COL2', 'COL1', 'COL2']
  print(list(t2.df.columns))  #> ['COL1', 'COL2', 'COL1_1', 'COL2_1']
  ```
 <hr class="sn-spacer-thin">
 ````
`````

<br>
<hr class="sn-spacer">

```{div} sn-phantom
&nbsp;
```
### *Working with SQL Scripts*

(intro/working-with-sql-scripts)=
`````{admonition} *Working with SQL Scripts*
:class: toggle, sn-gradient-header, sn-indent-h-cell-right-m, sn-toggle-expand, sn-incl-tabbed-shadow-b-blue

<hr class="sn-spacer">

````{tabbed} -
  [**snowmobile.Script**](./usage/script.md) accepts a full `path` to a sql file and parses its contents based on 
  patterns specified in [snowmobile.toml](./usage/snowmobile_toml).
+++  
  At a minimum, the file is split into individual st, each of which is 
  checked for decorated information in the form of a string directly preceding it
  wrapped in an opening (`/*-`) and closing (`-*/`) pattern, the simplest form of
  which is a single-line string that can be used as an accessor to the statement
  it precedes.
+++
  When no information is provided, [Script](./usage/script.ipynb) generates a 
  generic name for the statement based on the literal first SQL keyword 
  it contains and its index position.
````

````{tabbed} More Info
  Line **27** within *sample_table.sql* represents the minimum markup required to associate a 
  name with an individual statement; consistency in tag structure has obvious benefits, but
  this is a freeform string that can be anything.
+++
  Line **19** is an example of a special tag; the leading `qa-empty` tells 
  [**Script**](./usage/script.ipynb) to run assertion that its results are
  null (0 records) before continuing execution of the script.
+++
  The tags for statements beginning on lines **1**, **6**, and **17** were generated by 
  [**Script**](./usage/script.ipynb) based their contents and relative positions within the script.
````

```{div} sn-link-container
{link-badge}`./usage/script.html,cls=badge-primary text-white sn-usage,Usage: snowmobile.Script,tooltip=Intro & usage documentation for snowmobile.Script`
{link-badge}`./autoapi/snowmobile/core/script/index.html,cls=badge-secondary text-white sn-api,snowmobile.core.script,tooltip=API Documentation`
```

`````

````{admonition} Setup
:class: is-setup, sn-block-container, sn-indent-h-cell-m

```{div} sn-dedent-v-t-h, hanging
`path` is a full path ({class}`pathlib.Path` or {class}`str`) to a file, 
*sample_table.sql*, containing 5 standard sql st: 
```

```{literalinclude}  ./snippets/overview/sample_table.sql
:language: sql
:lines: 2-36
:lineno-start: 1
:emphasize-lines: 1, 6, 17, 20, 28
```
```{div} sn-snippet-trunc2, sn-float-right
[{fa}`file-code-o` sample_table.sql](./snippets.md#overview)
```

````

In [10]:
# -- setup --

# Verify table doesn't exist
if sn.exists('sample_table'):
    sn.drop('sample_table')

# Truncate 'any_other_table'
sn.truncate('any_other_table')

# Path to sample .sql file
from pathlib import Path

path = Path.cwd() / 'snippets' / 'overview' / 'sample_table.sql'

```{div} sn-pre-code
Given a `path` to *sample_table.sql*, a [Script](./usage/script) can be created with:
```

In [13]:
import snowmobile

script = snowmobile.Script(path=path, silence=True)

print(script)        #> snowmobile.Script('sample_table.sql')
print(script.depth)  #> 5
script.dtl()

sample_table.sql
1: Statement('create table~s1')
2: Statement('insert into~s2')
3: Statement('select data~s3')
4: Statement('qa-empty~verify 'sample_table' is distinct on 'col1'')
5: Statement('insert into~any_other_table')


```{div} sn-pre-code, sn-post-code
  Any [Statement](/autoapi/snowmobile/core/statement/index) can be accessed by 
  their {attr}`~snowmobile.core.Name.nm` or index position:
```

```{div} sn-pre-code, sn-post-code
  Statements can be accessed by their index position or name ({attr}`~snowmobile.core.Name.nm`):
```

In [51]:
script(5)                             #> Statement('insert into~any_other_table')
script(-1)                            #> Statement('insert into~any_other_table')
script('insert into~any_other_table') #> Statement('insert into~any_other_table')

Statement('insert into~any_other_table')

```{div} sn-pre-code, sn-post-code
Each [Statement](/autoapi/snowmobile/core/statement/index) has its own set of attributes:
```

In [20]:
s3 = script(3)  # store 3rd statement

print(s3.index)  #> 3
print(s3.sql)    #> select * from sample_table
print(s3.kw)     #> select
print(s3.desc)   #> sample_table
print(s3.nm)     #> select~sample_table

3
select
sample_table
select~sample_table
select * from sample_table
False
Empty DataFrame
Columns: []
Index: []


```{div} sn-pre-code, sn-post-code
Based on statement attributes, `script` can be subsetted and ran within that context:
```

```{div} sn-pre-code, sn-post-code
Based on statement attributes, `script` can be filtered and used within that context:
```

In [16]:
with script.filter(
    excl_desc=['.*any_other_table'],  # throwing out s5; pattern is regex not glob
    excl_kw=['select'],               # throwing out s3
) as s:
    s.run()

sample_table.sql
<1 of 3> create table~s1 (1s)......................................... <completed>
<2 of 3> insert into~s2 (0s).......................................... <completed>
<3 of 3> qa-empty~verify 'sample_table' is distinct on 'col1' (0s).... <passed>   


```{div} sn-pre-code, sn-post-code
Spans of statements are directly executable by index boundaries:
```

In [68]:
script.run((1, 3))

sample_table.sql
<1 of 6> create table~s1 (0s)............................... <completed>
<2 of 6> insert into~s2 (0s)................................ <completed>
<3 of 6> select data~s3 (0s)................................ <completed>


```{div} sn-pre-code, sn-post-code
And their results accessible retroactively:
```

In [48]:
script(3).results.head()

,col1,col2
0,1,1
1,2,4
2,3,9


```{div} sn-pre-code, sn-post-code
Generated SQL can be added directly to `script`:
```

In [60]:
# Generate some sql
drop_sql = sn.drop(nm='sample_table', obj='table', run=False)
print(drop_sql)      #> drop table if exists SAMPLE_TABLE

# Details (pre)
print(script.depth)  #> 5
print(script(-1))    #> Statement('insert into~any_other_table')

# Add statement
script.parse_one(s=drop_sql)

# Details (post)
print(script.depth)  #> 6
print(script(-1))    #> Statement('drop table~s6')

script.dtl()

sample_table.sql
1: Statement('create table~s1')
2: Statement('insert into~s2')
3: Statement('select data~s3')
4: Statement('qa-empty~verify 'sample_table' is distinct on 'col1'')
5: Statement('insert into~any_other_table')
6: Statement('drop table~s6')


```{div} sn-pre-code, sn-post-code
And descriptive statements can be skipped instead of commented out:
```

In [72]:
with script.filter(excl_kw=['select']) as s:
    s.run()

sample_table.sql
<1 of 5> create table~s1 (0s)............................... <completed>
<2 of 5> insert into~s2 (0s)................................ <completed>
<3 of 5> qa-empty~verify 'sample_table' is distinct on 'col1' (0s).... <passed>   
<4 of 5> insert into~any_other_table (0s)............................. <completed>
<5 of 5> drop table~s5 (0s)................................. <completed>


<hr class="sn-spacer">

```{div} sn-pre-code
See [Usage: Script](./usage/script.md) for more in-depth use of [snowmobile.Script](./usage/script).
```

<br>

### *Wrap-Up*
<hr class="sn-green-thick sn-indent-h-cell-right-m">

`````{admonition} Note
:class: note, sn-indent-h-cell-m

 ```{div} sn-dedent-v-b-h
 By instantiating `t1` and `script` with the same instance of [{fa}`fixture sn`](fixture-sn),
 **the same instance of {xref}`SnowflakeConnection` and [Configuration](./usage/snowmobile_toml) 
 is shared amongst:**
 ```
 - {class}`sn:` {class}`~snowmobile.Snowmobile`
 - {class}`t1:` {class}`~snowmobile.Table`
 - {class}`script:` {class}`~snowmobile.Script`

`````

# NO RENDER BELOW THIS POINT
---

All cells below this are either excluded from output via the `remove-cell` cell-tag
or contain contents that will not visibly render in the output.

## Style


```css
<style>
.md-typeset h1, .md-typeset h2 {
    margin-top: -0.5rem;
}
    
.md-typeset h3 {
    margin-top: -0.5rem;
    margin-bottom: 0.2rem;
    font-size: 140%;
}
    
/* .md-typeset .admonition.is-setup {
    border-left-color: #11838e;
    margin: -0.15rem 1.5rem .7rem 0.85rem;
} */
</style>
```

<style>
.md-typeset h1, .md-typeset h2 {
    margin-top: -0.5rem;
}
    
.md-typeset h3 {
    margin-top: -0.5rem;
    margin-bottom: 0.2rem;
    font-size: 140%;
}
    
<!-- .md-typeset .admonition.is-setup { -->
<!--     border-left-color: #11838e; -->
<!--     margin: -0.15rem 1.5rem .7rem 0.85rem; -->
<!--     display: block; -->
}
</style>